In [11]:
!pip install PyPDF2 langchain sentence-transformers faiss-cpu pandas


In [15]:


# Import necessary modules
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline
from google.colab import files

# Step 1: Extract text from a PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Step 2: Chunk the extracted text
def chunk_text(text, chunk_size=500, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)

# Step 3: Embed chunks using Sentence Transformers
def embed_text(chunks, model):
    return model.encode(chunks)

# Step 4: Add embeddings to FAISS vector store
def create_faiss_index(embeddings):
    dimension = embeddings[0].shape[0]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index

# Step 5: Search for similar chunks
def search_index(index, query_vector, k=5):
    distances, indices = index.search(query_vector, k)
    return indices, distances

# Step 6: Use a free LLM to generate responses
def generate_response(retrieved_chunks, query, qa_model):
    context = " ".join(retrieved_chunks)
    prompt = f"Based on the following information, answer the question: {query}\n\n{context}"
    response = qa_model(prompt)
    return response[0]['generated_text']

# Main script
if __name__ == "__main__":
    # Provide the PDF file path
    uploaded = files.upload()  # Manually upload the PDF
    pdf_path = list(uploaded.keys())[0]

    # Step 1: Extract text
    pdf_text = extract_text_from_pdf(pdf_path)

    # Step 2: Chunk the text
    chunks = chunk_text(pdf_text)

    # Step 3: Generate embeddings
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = embed_text(chunks, embedding_model)

    # Step 4: Create FAISS index
    index = create_faiss_index(embeddings)

    # Step 5: Handle user query
    user_query = input("Enter query :")
    query_vector = embedding_model.encode([user_query])
    indices, distances = search_index(index, query_vector)
    retrieved_chunks = [chunks[i] for i in indices[0]]
    print("Relevant chunks retrieved:", retrieved_chunks)

    # Step 6: Generate response using a free LLM
    qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")
    response = generate_response(retrieved_chunks, user_query, qa_pipeline)
    print("Response:", response)


Saving Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life - 2017-2018.pdf to Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life - 2017-2018 (7).pdf
Text extracted from PDF: Tables, Charts, and 
Graphs 
with Examples from History, Economics, 
Education, Psychology, Urban Affairs and Everyday Life
REVISED: MICHAEL LOLKUS 2018Tables, Charts, and 
Graphs BasicsWe use charts and graphs to visualize data.  
This data can either be generated data, data gathered from 
an experiment, or data collected from some source.
A picture tells a thousand words so it is not a surprise that many people use charts and graphs when explaining data.Types of Visual 
Representations of D
Enter query :From page 6 get the tabular data 
Relevant chunks retrieved: ['Representations of DataTable of Yearly U.S. GDP by \nIndustry (in millions of dollars)\nYear 2010 2011 2012 20

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Token indices sequence length is longer than the specified maximum sequence length for this model (642 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Response: 2010 2011 2012 2013 2014 2015 All Industries 26093515 27535971 286632
